In [2]:
import subprocess
import csv
import json

# Inicializa acumuladores para o resumo
total_timeslots = 0
total_eprs_created = 0
total_eprs_consumed = 0
total_fidelity = 0.0
fidelity_count = 0

# Caminho do arquivo CSV
csv_file_path = "link_metrics_output.csv"

# Número de sucessos desejados
success_target = 3  # Número desejado de sucessos

# Inicializa contadores
success_count = 0
attempt_count = 0

# Loop para rodar a simulação até o número desejado de sucessos
while success_count < success_target:
    attempt_count += 1
    try:
        # Executa o script simulation.py em um novo processo, passando o índice da simulação como argumento
        result = subprocess.run(
            ["python", "script.py", str(attempt_count)],
            capture_output=True,
            text=True,
            check=True
        )

        # Filtra a saída para tentar capturar apenas o JSON
        output = result.stdout.strip()

        # Exibe a inicialização das variáveis
        print(f"Tentativa {attempt_count}")
        print("Saída obtida: Hosts inicializados")
        print("Canais inicializados")
        print("Pares EPRs adicionados")

        # Filtra a parte do JSON
        json_str = ""
        try:
            # Encontra o JSON na saída
            start_idx = output.index('{')
            end_idx = output.rindex('}') + 1
            json_str = output[start_idx:end_idx]
        except ValueError:
            print(f"Tentativa {attempt_count} - Saída não contém JSON válido: {output}")
            continue
        
        # Verifica se a saída é um JSON válido
        try:
            simulation_result = json.loads(json_str)
            # Exibe o JSON com formatação bonita
            print(json.dumps(simulation_result, indent=4))
        except json.JSONDecodeError:
            print(f"Tentativa {attempt_count} - Erro ao processar JSON: {output}")
            continue
        
        # Verifica o sucesso da simulação
        if simulation_result.get("success", False):
            success_count += 1
            print(f"Simulação {attempt_count} bem-sucedida! (Sucessos: {success_count})")
        
        # Verifica se o número de sucessos desejado foi alcançado
        if success_count >= success_target:
            print(f"Alcançado o número de sucessos desejados ({success_target}). Finalizando simulações.")
            break  # Encerra o loop quando o número de sucessos for atingido

    except subprocess.CalledProcessError as e:
        print(f"Erro na execução da simulação {attempt_count}: {e}")
        continue

# Processa o arquivo CSV para coletar as métricas
try:
    with open(csv_file_path, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Pula o cabeçalho
        
        for row in reader:
            # Verifica se a linha tem o número esperado de colunas
            if len(row) < 6:
                print(f"Linha inválida no CSV: {row}")
                continue
            
            try:
                # Processa as métricas corretamente
                timeslot_total = int(row[2])
                eprs_created = int(row[3])
                eprs_consumed = int(row[4])
                fidelity_avg_raw = row[5]

                # Deserializa a fidelidade média
                fidelity_avg = 0.0
                try:
                    fidelity_avg = json.loads(fidelity_avg_raw.replace("'", "\""))
                    if isinstance(fidelity_avg, dict) and 'media' in fidelity_avg:
                        fidelity_avg = fidelity_avg['media']
                    else:
                        fidelity_avg = 0.0
                except json.JSONDecodeError:
                    fidelity_avg = 0.0

                # Atualiza as métricas acumuladas
                total_timeslots += timeslot_total
                total_eprs_created += eprs_created
                total_eprs_consumed += eprs_consumed
                if isinstance(fidelity_avg, (int, float)) and fidelity_avg > 0:
                    total_fidelity += fidelity_avg
                    fidelity_count += 1
            except (IndexError, ValueError) as e:
                print(f"Erro ao processar a linha: {row}. Erro: {e}")
                continue

except FileNotFoundError:
    print(f"Arquivo {csv_file_path} não encontrado.")
except Exception as e:
    print(f"Erro ao processar o arquivo CSV: {e}")

# Calcula a fidelidade média total
average_fidelity = total_fidelity / fidelity_count if fidelity_count > 0 else 0.0

# Exibe o resumo das métricas no final
print("\nResumo das Métricas da Simulação (após todas as tentativas):")
print(f"Total de Timeslots: {total_timeslots}")
print(f"Total de EPRs Criados: {total_eprs_created}")
print(f"Total de EPRs Consumidos: {total_eprs_consumed}")
print(f"Fidelidade Média Total: {average_fidelity}")


Erro na execução da simulação 1: Command '['python', 'script.py', '1']' returned non-zero exit status 1.
Erro na execução da simulação 2: Command '['python', 'script.py', '2']' returned non-zero exit status 1.
Erro na execução da simulação 3: Command '['python', 'script.py', '3']' returned non-zero exit status 1.
Erro na execução da simulação 4: Command '['python', 'script.py', '4']' returned non-zero exit status 1.
Erro na execução da simulação 5: Command '['python', 'script.py', '5']' returned non-zero exit status 1.
Erro na execução da simulação 6: Command '['python', 'script.py', '6']' returned non-zero exit status 1.
Erro na execução da simulação 7: Command '['python', 'script.py', '7']' returned non-zero exit status 1.
Erro na execução da simulação 8: Command '['python', 'script.py', '8']' returned non-zero exit status 1.
Erro na execução da simulação 9: Command '['python', 'script.py', '9']' returned non-zero exit status 1.
Tentativa 10
Saída obtida: Hosts inicializados
Canais i

KeyboardInterrupt: 